In [1]:
#Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load in student data and examine
student_df = pd.read_csv('Sample Data for Analysis.xlsx - Raw Data.csv')
student_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18407 entries, 0 to 18406
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STUDENTS       18407 non-null  int64  
 1   Campus         18407 non-null  object 
 2   Degree Level   18407 non-null  object 
 3   CIP Code       18407 non-null  float64
 4   Program Title  18407 non-null  object 
 5   City           18407 non-null  object 
 6   State          18407 non-null  object 
 7   GEOID          18280 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.1+ MB


In [3]:
#Count number of missing values for GEOID
student_df['GEOID'].value_counts(dropna=False)[np.nan]

127

In the Student dataset, we are missing 127 values of GEOID. This is about .7% of the data. This will be addressed in the following steps.

In [4]:
#Assemble a new dataframe that finds the most frequent GEOID for each city-state pair.
#This will be used to impute missing GEOIDs
city_geoid_df = student_df[['City','State','GEOID']]
city_common_geoid_df = city_geoid_df.groupby(['City','State']).GEOID.agg(pd.Series.mode).to_frame()

In [5]:
#examine common GEOIDs
city_common_geoid_df['GEOID'] = city_common_geoid_df['GEOID'][0]
city_common_geoid_df

,,GEOID
City,State,
ABILENE,KS,2.004108e+10
ADDISON,TX,2.004108e+10
ADRIAN,MO,2.004108e+10
AGENCY,MO,2.004108e+10
ALAMEDA,CA,2.004108e+10
...,...,...
WRIGHT CITY,MO,2.004108e+10
YORK,PA,2.004108e+10
YUMA,AZ,2.004108e+10


In [6]:
#Merge the student dataframe with the GEOID mode dataframe
df = student_df.merge(city_common_geoid_df, how='left', left_on=['City', 'State'], right_on=['City', 'State'])
df = df.rename(columns={'GEOID_x': 'GEOID', 'GEOID_y': 'City_GEOID_mode'})

In [7]:
#Count of nan values before imputation
df['GEOID'].value_counts(dropna=False)[np.nan]

127

In [8]:
#Impute missing GEOID values with the highest frequency GEOID per city-state pair
df['GEOID'].fillna(value=df['City_GEOID_mode'], inplace=True)

In [9]:
#Count of na values after GEOID imputation
na_values = df[df['GEOID'] == np.nan]
len(na_values)

0

We have now handled the NA values from this source.

In [10]:
#Read distance csv
distance_df = pd.read_csv('Sample Data for Analysis.xlsx - Distances to Campus.csv')
distance_df.head()

,GEOID,Distance,Campus
0,29105960400,1.93467,East Campus
1,29105960500,2.19978,East Campus
2,29105960300,2.89078,East Campus
3,29105960600,3.53737,East Campus
4,29105960100,9.27136,East Campus


In [11]:
#map campus names for join with the main dataframe df
distance_df['Campus'] = distance_df['Campus'].map({'East Campus':'ETC','Main Campus':'MNC','West Campus':'WTC', 'Online':'WEB'})
distance_df.head()

,GEOID,Distance,Campus
0,29105960400,1.93467,ETC
1,29105960500,2.19978,ETC
2,29105960300,2.89078,ETC
3,29105960600,3.53737,ETC
4,29105960100,9.27136,ETC


In [12]:
#Merge dataframe with the distance table. This associates each student with their distance from campus.
#The students that didn't have a GEOID associated with them were assigned the most frequent
#GEOID in their city-state pair.
combined_df = df.merge(distance_df, how='left',left_on=['GEOID', 'Campus'], right_on=['GEOID', 'Campus'])
#check the number of na values after merging. This would attribute to GEOIDs that don't have a distance mapping
len(combined_df[combined_df['Distance'].isna()])

5493

In [13]:
null_df = combined_df[combined_df['Distance'].isna()]
null_df['x'] = null_df['Distance'].isnull()
null_df = null_df.groupby('State')['x'].sum()
null_df

State
AE       4
AK       4
AL       2
AR      47
AZ      10
CA      20
CO      12
FL      21
GA      14
HI       2
IA       2
ID       4
IL      15
IN       3
KS      15
KY       4
LA       5
MD       5
MI       5
MN       2
MO    5187
MS       4
MT       2
NC       8
NE       1
NH       1
NJ       3
NM       2
NV       4
NY       9
OH       5
OK       9
OR       6
PA       2
SC       2
SD       1
TN       2
TX      28
UT       1
VA       8
VT       1
WA       8
WI       2
WY       1
Name: x, dtype: int64

From this, we find that we still have a lot of GEOIDs that do not have distance listed. These are attributed to online students. Since the majority of these students still live in MO, we will attempt to solve this by calculating the average distance for each city-state pair, and then assign the average distance to the missing distances for that particular city.

This method will use average distance from a campus based on the student's city-state pair.

In [14]:
#Calculate average distance in each city-state pair
avg_distance_df = combined_df.groupby(['City', 'State'])['Distance'].mean().to_frame()
avg_distance_df = avg_distance_df.rename(columns={'Distance':'avg_distance'})
avg_distance_df

,,avg_distance
City,State,
ABILENE,KS,NaN
ADDISON,TX,357.8410
ADRIAN,MO,94.2929
AGENCY,MO,185.5900
ALAMEDA,CA,NaN
...,...,...
WRIGHT CITY,MO,163.6880
YORK,PA,NaN
YUMA,AZ,NaN


In [15]:
combined_df = combined_df.merge(avg_distance_df, left_on=['City','State'], right_on=['City','State'])
combined_df['Distance'].fillna(value=combined_df['avg_distance'], inplace=True)
combined_df

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID,City_GEOID_mode,Distance,avg_distance
0,15900,WEB,CT,52.0401,"Admin. Assistant, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679
1,11676,MNC,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,8.594920,6.038679
2,14013,WEB,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679
3,14678,WEB,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679
4,574,ETC,AAS,49.0102,Airline/Commercial/Prof. Pilot/Flight Crew,SPRINGFIELD,MO,2.907700e+10,2.004108e+10,53.743200,6.038679
...,...,...,...,...,...,...,...,...,...,...,...
18402,8841,MNC,AAS,48.0508,Welding Technology/Welder,MAYSVILE,MO,2.906308e+10,2.004108e+10,193.600000,193.600000
18403,9248,MNC,AAS,48.0508,Welding Technology/Welder,GRANGER,IN,1.803900e+10,2.004108e+10,NaN,NaN
18404,10646,MNC,CT,48.0508,Welding Technology/Welder,FOREST HILL,TX,4.843911e+10,2.004108e+10,387.251000,387.251000
18405,13393,WTC,CT,48.0508,Welding Technology/Welder,FT LEONARDWOOD,MO,2.916947e+10,2.004108e+10,5.885360,5.885360


In [16]:
#Drop remaining Nan values from combined_df
print(len(combined_df))
combined_df = combined_df.dropna()
print(len(combined_df))

18407
18063


After replacing the missing distances with the average distance in each city, we were able to reduce the Nan values from 5493 to 344. These remaining can be dropped without significant impact to the analysis.

In [17]:
binned_df = combined_df
bins = [0,20,40,60,80,100,120,140,160,180,200,220,240,260,280,300,10000]
labels = ['0-20','20-40','40-60','60-80','80-100','100-120','120-140','140-160','160-180','180-200','200-220','220-240','240-260','260-280','280-300','>300']
binned_df['Miles'] = pd.cut(binned_df['Distance'], bins=bins, labels=labels, include_lowest=True).to_frame()
binned_df

,STUDENTS,Campus,Degree Level,CIP Code,Program Title,City,State,GEOID,City_GEOID_mode,Distance,avg_distance,Miles
0,15900,WEB,CT,52.0401,"Admin. Assistant, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679,0-20
1,11676,MNC,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,8.594920,6.038679,0-20
2,14013,WEB,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679,0-20
3,14678,WEB,AAS,1.0000,"Agriculture, General",SPRINGFIELD,MO,2.907700e+10,2.004108e+10,6.038679,6.038679,0-20
4,574,ETC,AAS,49.0102,Airline/Commercial/Prof. Pilot/Flight Crew,SPRINGFIELD,MO,2.907700e+10,2.004108e+10,53.743200,6.038679,40-60
...,...,...,...,...,...,...,...,...,...,...,...,...
18400,6687,MNC,CT,48.0508,Welding Technology/Welder,SEARCY,AR,5.145070e+09,2.004108e+10,157.043000,157.043000,140-160
18401,7944,MNC,AAS,48.0508,Welding Technology/Welder,VAN BUREN,MO,2.903596e+10,2.004108e+10,121.220000,121.220000,120-140
18402,8841,MNC,AAS,48.0508,Welding Technology/Welder,MAYSVILE,MO,2.906308e+10,2.004108e+10,193.600000,193.600000,180-200
18404,10646,MNC,CT,48.0508,Welding Technology/Welder,FOREST HILL,TX,4.843911e+10,2.004108e+10,387.251000,387.251000,>300


In [18]:
binned_df = binned_df.drop(columns=['Degree Level','CIP Code', 'Program Title', 'City', 'State', 'GEOID', 'City_GEOID_mode','Distance','avg_distance'])
binned_df

,STUDENTS,Campus,Miles
0,15900,WEB,0-20
1,11676,MNC,0-20
2,14013,WEB,0-20
3,14678,WEB,0-20
4,574,ETC,40-60
...,...,...,...
18400,6687,MNC,140-160
18401,7944,MNC,120-140
18402,8841,MNC,180-200
18404,10646,MNC,>300


In [19]:
pivot_df = binned_df.pivot_table(columns='Campus', values='STUDENTS', index='Miles', aggfunc='count')
pivot_df = pivot_df.reindex(labels)
pivot_df = pivot_df.fillna(value=0)
total = len(combined_df)
for col in pivot_df.columns:
    pivot_df[col] = pivot_df[col]/total
pivot_df

Campus,ETC,MNC,WEB,WTC
Miles,,,,
0-20,0.032553,0.447157,0.155345,0.028954
20-40,0.010629,0.078835,0.051431,0.003100
40-60,0.002159,0.038476,0.056303,0.000111
60-80,0.001550,0.015280,0.007418,0.000664
80-100,0.000498,0.008913,0.002713,0.000111
100-120,0.000221,0.008083,0.002436,0.000111
120-140,0.000166,0.008803,0.002713,0.000055
140-160,0.000166,0.006034,0.002270,0.000000
160-180,0.000000,0.006976,0.001993,0.000000


In [20]:
pivot_df['Total'] = pivot_df['ETC'] + pivot_df['MNC'] + pivot_df['WEB'] + pivot_df['WTC']
pivot_df

Campus,ETC,MNC,WEB,WTC,Total
Miles,,,,,
0-20,0.032553,0.447157,0.155345,0.028954,0.664009
20-40,0.010629,0.078835,0.051431,0.003100,0.143996
40-60,0.002159,0.038476,0.056303,0.000111,0.097049
60-80,0.001550,0.015280,0.007418,0.000664,0.024913
80-100,0.000498,0.008913,0.002713,0.000111,0.012235
100-120,0.000221,0.008083,0.002436,0.000111,0.010851
120-140,0.000166,0.008803,0.002713,0.000055,0.011737
140-160,0.000166,0.006034,0.002270,0.000000,0.008470
160-180,0.000000,0.006976,0.001993,0.000000,0.008969


In [21]:
pivot_df.to_csv('pivot_frame.csv')

<h1>Assumptions to Address</h1>
There were two main problems to handle during this analysis; 1.) How do you handle students without a GEOID, and 2.) How do you handle GEOIDs that don't have mapped distances. Following, I will address how I approached each of these.

<h3>Students with missing GEOIDs</h3>
I approached this by counting the occurences of GEOIDs in each city-state pair, and creating a dataframe assigning the mode of the GEOID to each city-state pair. This is done in cell 4. Then, I joined the original dataframe with this GEOID mode dataframe, and filled the NA values of student GEOID with the GEOID mode.

<h3>GEOIDs with missing distances</h3>
This was a far more pervasive problem than the previous, affecting nearly 5500 students, largely due to online students (see below). I approached this problem by calculating the average distance from campus for each city-state pair, creating a dataframe holding each city and state with it's associated average distance. Then, I joined the student dataframe with this distance dataframe on City and State and imputed the average distance on the missing distances.

<h3>Online Students</h3>
My first thought was that the distances for online students was not something to be considered, but upon thinking on it more, and due to the locations of many of the online students, this could not be done. The majority of the online students live in Springfield, or MO, for that matter. My assumption is that being an online student does not preclude being a student that visits a campus, and while I don't know the details of the dataset or the use case of this analysis, I found it desirable to also include online students' locations. 

<h1>Improvements</h1>

Looking forward, there are a few improvements to be made:
    
1.) Higher Precision bins could be used. Particularly useful in the nearby radii, smaller bins would tell a deeper story about have students are distributed around each campus.
    
2.) GEOIDs to be imputed could come from the distribution of GEOIDs. So, instead of just considering the mode, a GEOID could be chosen by a random number generator using the value counts of GEOID in each city-state combination. This could lead to higher accuracy during the binning process. 
    
3.) Similar to the improvement above, instead of just using the average distance in each city, a random generator could be used to pick a distance based on the value counts in each city. This would also allow for a deeper understanding of where students without distances could be coming from.